# Pipelines

Pipelines enable us to define the flow of data from the initial dataset to a trained and scored model. The parameters of the pipeline can be tweaked and also optimized using algormithms like grid-search.

In [9]:
import sys
sys.path.append('../scripts')

import numpy as np
import pandas as pd
import helpers_models as hm
import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

First, load the data.

In [2]:
data = hm.load_pivot_numbers()

In [3]:
first_week, last_week = data['tweets'].columns.min(), data['tweets'].columns.max()

In [4]:
data = hm.make_target(data, last_week)

Split the data into train and test.

In [5]:
data_train, data_test = train_test_split(data)

Now make a pipeline.

In [8]:
pipeline = Pipeline([
    # ('normal', transforms.Normalizer()), # makes errors
    ('decay', transforms.TimeDecayApplier(first_week=first_week, target_week=last_week)),
    ('logreg', LogisticRegressionCV(max_iter=200, n_jobs=-1, verbose=2))
])

The next step is to train the pipeline.

In [10]:
%%time
pipeline.fit(data_train.drop('target', axis=1, level=0), data_train['target'])

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  4.2min finished


CPU times: user 2min 3s, sys: 1min 36s, total: 3min 39s
Wall time: 7min 38s


Pipeline(steps=[('decay', TimeDecayApplier(first_week=23, target_week=36)), ('logreg', LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=200,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=2))])

Finally, use the pipeline to predict test values and scores.

In [12]:
%%time
predicted = pipeline.predict(data_test.drop('target', axis=1, level=0))
report = classification_report(data_test['target'], predicted)
print(report)

             precision    recall  f1-score   support

      False       0.91      0.99      0.95   1826596
       True       0.76      0.25      0.37    238812

avg / total       0.89      0.90      0.88   2065408

CPU times: user 27.8 s, sys: 25.4 s, total: 53.2 s
Wall time: 53.1 s
